In [ ]:
"""
定义一个非常简单的Dense的模拟层来模拟Dense的功能，串联Dense一起来使用。
详细的Dense的定义在这里:
https://github.com/keras-team/keras/blob/master/keras/layers/core.py#L796
如果要建立更复杂的layer，可以参考keras的源代码。
"""

from tensorflow.keras import backend as K
from tensorflow.keras.layers import Layer
from tensorflow.keras.layers import Dense
import pandas as pd
from tensorflow.keras.models import Sequential

# 这玩意怎么测试？
class MyDenseLayer(Layer):
    def __init__(self, output_dim, **kwargs):
        print("output_dim - {}".format(output_dim))
        self.output_dim = output_dim
        self.activation = K.sigmoid
        super(MyDenseLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        # 定义了一个可训练的权重矩阵。input_shape在定义层中input_shape[0]是batch_size
        print("input_shape - {}".format(input_shape))
        self.kernel = self.add_weight(name='kernel',
                                      shape=(input_shape[1], self.output_dim),
                                      initializer='uniform',
                                      trainable=True)
        super(MyDenseLayer, self).build(input_shape)  # Be sure to call this at the end

    def call(self, x):
        print("x - {}".format(x))
        output = K.dot(x, self.kernel)
        if self.activation is not None:
            output = self.activation(output)
        print("output - {}".format(output))
        return output

    def compute_output_shape(self, input_shape):
        assert input_shape and len(input_shape) >= 2
        assert input_shape[-1]
        output_shape = list(input_shape)
        output_shape[-1] = self.units
        return tuple(output_shape)


# 1. 读取数据
first_train_line = 500
second_train_line = 700

pima_data = pd.read_csv("./input_data/diabetes.csv")
pima_data.info()
first_train_data = pima_data[0:first_train_line]
second_train_data = pima_data[first_train_line:second_train_line]
test_data = pima_data[second_train_line:]

X_1_train = first_train_data.iloc[:, 0:8]
Y_1_train = first_train_data.iloc[:, 8]
X_2_train = second_train_data.iloc[:, 0:8]
Y_2_train = second_train_data.iloc[:, 8]
X_test = test_data.iloc[:, 0:8]
Y_test = test_data.iloc[:, 8]

np_X_1_train = X_1_train.to_numpy()
np_Y_1_train = Y_1_train.to_numpy()
np_X_2_train = X_2_train.to_numpy()
np_Y_2_train = Y_2_train.to_numpy()
np_X_test = X_test.to_numpy()
np_Y_test = Y_test.to_numpy()

# 2. 使用定制的layer
model = Sequential()
model.add(MyDenseLayer(6, input_shape=(8,)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

model.fit(np_X_1_train, np_Y_1_train, batch_size=16, epochs=20, verbose=1, validation_split=0.2)
print("finish")
